In [1]:
essay = '''TensorFlow: A Versatile Framework for Deep Learning
TensorFlow is an open-source machine learning framework developed by the Google Brain team.
Renowned for its flexibility and scalability, TensorFlow has become one of the most popular tools for deep learning research and production-level deployment.
At its core, TensorFlow offers a computational graph framework that allows developers to define and execute complex machine learning models efficiently.
The framework is designed to handle a wide range of tasks, from image and natural language processing to reinforcement learning and more.

Usage of TensorFlow: Versatility and Wide Adoption

One of TensorFlow's standout features is its versatility. It provides a unified platform for creating and training machine learning models across diverse domains.
TensorFlow supports both traditional neural networks and cutting-edge deep learning architectures like convolutional neural networks (CNNs) and recurrent neural networks (RNNs).
Furthermore, TensorFlow's ecosystem includes high-level APIs like Keras, which simplifies the process of building and training deep learning models,
making it accessible to both beginners and experts.

Keras Integration: Streamlined Model Development

Keras, a high-level neural networks API, has been tightly integrated into TensorFlow since version 2.0.
This integration offers users the best of both worlds—Keras's user-friendly interface and TensorFlow's power and scalability.
Keras allows developers to define neural network architectures using a simple and intuitive syntax. It abstracts away many of the low-level details,
enabling rapid prototyping and experimentation. By using Keras within TensorFlow, developers can seamlessly transition from model design and training to production deployment.

Conclusion: Empowering Innovation in Machine Learning

In conclusion, TensorFlow, with its integrated Keras API, empowers developers and researchers in the field of machine learning.
Its versatility and wide adoption make it a preferred choice for a variety of applications, from image classification to natural language understanding.
Whether you're just starting your journey in deep learning or working on advanced research, TensorFlow's rich ecosystem, coupled with Keras's user-friendly interface,
provides the tools and flexibility needed to push the boundaries of innovation in the field of machine learning.'''

In [58]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer

In [5]:
tokenizer = Tokenizer()

In [6]:
tokenizer.fit_on_texts([essay])

In [34]:
len(tokenizer.word_index) # To see what index has been assigned to each word

173

In [15]:
input_sequences = []
for sentence in essay.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
  for i in range(1, len(tokenized_sentence)):
    n_gram = tokenized_sentence[:i+1]
    input_sequences.append(n_gram)

In [23]:
max_len = max([len(x) for x in input_sequences])

In [27]:
from keras.preprocessing.sequence import pad_sequences
padded_sequences = pad_sequences(input_sequences, maxlen = max_len, padding = 'pre')

In [31]:
X = padded_sequences[:,:-1]
y = padded_sequences[:,-1]

In [33]:
X.shape, y.shape

((318, 22), (318,))

In [40]:
from keras.utils import to_categorical

y = to_categorical(y, num_classes = 174)
y.shape

In [42]:
#Architecture of model: I will be using 3 layers in the NN.
#1st layer = Embedding Layer: It will a dense vector
#2nd layer = LSTM layer
#3rd layer = Dense Layer for output

In [43]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [52]:
model = Sequential()
model.add(Embedding(174, 100, input_length = 22))
model.add(LSTM(128))
model.add(Dense(174, activation = 'softmax'))

In [53]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 100)           17400     
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 174)               22446     
                                                                 
Total params: 157094 (613.65 KB)
Trainable params: 157094 (613.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs = 100)

In [71]:
text = "what is tensorflow"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_text = pad_sequences([token_text], maxlen = 22, padding = 'pre')
  # predict
  pos = np.argmax(model.predict(padded_text))

  for word, index in tokenizer.word_index.items():
    if index==pos:
      # print(word)
      text = text+' '+word
      print(text)

1/1 [==============================] - 0s 40ms/step
what is tensorflow is
1/1 [==============================] - 0s 37ms/step
what is tensorflow is an
1/1 [==============================] - 0s 36ms/step
what is tensorflow is an open
1/1 [==============================] - 0s 39ms/step
what is tensorflow is an open source
1/1 [==============================] - 0s 47ms/step
what is tensorflow is an open source machine
1/1 [==============================] - 0s 39ms/step
what is tensorflow is an open source machine learning
1/1 [==============================] - 0s 35ms/step
what is tensorflow is an open source machine learning framework
1/1 [==============================] - 0s 29ms/step
what is tensorflow is an open source machine learning framework developed
1/1 [==============================] - 0s 24ms/step
what is tensorflow is an open source machine learning framework developed by
1/1 [==============================] - 0s 23ms/step
what is tensorflow is an open source machine learnin

In [72]:
import pickle

pickle.dump(model, open("model.pickle", 'wb'))